In [ ]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split


df = pd.read_table('Training.txt', encoding = 'latin-1',lineterminator='\n',header = 0 ,error_bad_lines=False )
df.head()

train, test = train_test_split(df, test_size=0.2)

train = train.replace('+','positive')
train = train.replace('-','negative' )


train['Sentiment1'] = train.Sentiment1.astype(str)

data = train.iloc[:,2]
data.replace("[^a-zA-Z]"," ", regex=True, inplace=True)

data = data.str.lower()
data.head()



contractions = pd.read_csv('/Users/vineethkumarrenukuntla/Downloads/contractions.csv', index_col='Contraction')
contractions.index = contractions.index.str.lower()
contractions.Meaning = contractions.Meaning.str.lower()
contractions_dict = contractions.to_dict()['Meaning']
# Defining regex patterns.
urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|(www\.)[^ ]*)"
userPattern       = '@[^\s]+'
hashtagPattern    = '#[^\s]+'
alphaPattern      = "[^a-z0-9<>]"
sequencePattern   = r"(.)\1\1+"
seqReplacePattern = r"\1\1"

# Defining regex for emojis
smileemoji        = r"[8:=;]['`\-]?[)d]+"
sademoji          = r"[8:=;]['`\-]?\(+"
neutralemoji      = r"[8:=;]['`\-]?[\/|l*]"
lolemoji          = r"[8:=;]['`\-]?p+"

def preprocess_apply(tweet):

    tweet = tweet.lower()

    # Replace all URls with '<url>'
    tweet = re.sub(urlPattern,'<url>',tweet)
    # Replace @USERNAME to '<user>'.
    tweet = re.sub(userPattern,'<user>', tweet)
    
    # Replace 3 or more consecutive letters by 2 letter.
    tweet = re.sub(sequencePattern, seqReplacePattern, tweet)

    # Replace all emojis.
    tweet = re.sub(r'<3', '<heart>', tweet)
    tweet = re.sub(smileemoji, '<smile>', tweet)
    tweet = re.sub(sademoji, '<sadface>', tweet)
    tweet = re.sub(neutralemoji, '<neutralface>', tweet)
    tweet = re.sub(lolemoji, '<lolface>', tweet)

    for contraction, replacement in contractions_dict.items():
        tweet = tweet.replace(contraction, replacement)

    # Remove non-alphanumeric and symbols
    tweet = re.sub(alphaPattern, ' ', tweet)

    # Adding space on either side of '/' to seperate words (After replacing URLS).
    tweet = re.sub(r'/', ' / ', tweet)
    return tweet

%%time
data['processed_text'] = data.Tweet.apply(preprocess_apply)



from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

cv = CountVectorizer(ngram_range=(1,1))
traindataset= cv.fit_transform(data.values.astype('U'))

rm = RandomForestClassifier(n_estimators=100, criterion='entropy')
rm.fit(traindataset,train['Sentiment1'])

datat = test.iloc[:,2]
datat.replace("[^a-zA-Z]"," ", regex=True, inplace=True)
datat = datat.str.lower()
datat.head()

testdataset= cv.transform(datat.values.astype('U'))
prediction = rm.predict(testdataset)

prediction

dft1 = pd.read_table('test1_public.txt', encoding = 'latin-1',skiprows = [77595])

datat = dft1.iloc[:,1]
datat.replace("[^a-zA-Z]"," ", regex=True, inplace=True)
datat = datat.str.lower()
datat.head()



testdataset= cv.transform(datat.values.astype('U'))
prediction1 = rm.predict(testdataset)
prediction1

dff1 = pd.DataFrame(prediction1)
dff1

dff1 = dff1.replace('positive','+')
dff1 = dff1.replace('negative','-')
dff1

dff1.to_csv('result1.csv')

dft2 = pd.read_table('test2_public.txt', encoding = 'latin-1')
dft2.head()

datat = dft2.iloc[:,1]
datat.replace("[^a-zA-Z]"," ", regex=True, inplace=True)
datat = datat.str.lower()
datat.head()

testdataset= cv.transform(datat.values.astype('U'))
prediction3 = rm.predict(testdataset)

dff2 = pd.DataFrame(prediction3)

dff2 = dff2.replace('positive','+')
dff2 = dff2.replace('negative','-')
dff2

dff2.to_csv('testresult2.csv')
